<a href="https://colab.research.google.com/github/Manohar6143/Manohar/blob/main/Siamese_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Install dependencies
!pip install torchvision --quiet

# 2. Imports
import torch, torch.nn as nn, torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import random

# 3. Dataset with Pairs
class SiameseDataset(Dataset):
    def __init__(self, mnist):
        self.mnist = mnist
        self.data = mnist.data
        self.targets = mnist.targets

    def __getitem__(self, index):
        img1 = self.data[index]
        label1 = self.targets[index]

        should_match = random.randint(0, 1)
        if should_match:
            idx2 = (self.targets == label1).nonzero()[random.randint(0, 100)][0]
        else:
            idx2 = (self.targets != label1).nonzero()[random.randint(0, 100)][0]

        img2 = self.data[idx2]
        label = torch.tensor([int(label1 == self.targets[idx2])], dtype=torch.float32)

        return img1.unsqueeze(0).float()/255, img2.unsqueeze(0).float()/255, label

    def __len__(self):
        return len(self.mnist)

transform = transforms.ToTensor()
mnist = datasets.MNIST(root='.', train=True, download=True, transform=transform)
trainset = SiameseDataset(mnist)
trainloader = DataLoader(trainset, batch_size=32, shuffle=True)

# 4. Siamese Model
class SiameseNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, 3), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 64, 3), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.fc = nn.Linear(64*5*5, 128)

    def forward_once(self, x):
        x = self.cnn(x)
        return self.fc(x.view(x.size(0), -1))

    def forward(self, x1, x2):
        emb1 = self.forward_once(x1)
        emb2 = self.forward_once(x2)
        return F.pairwise_distance(emb1, emb2)

# 5. Training
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SiameseNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

def contrastive_loss(dist, label, margin=1.0):
    return (1 - label) * dist**2 + label * torch.clamp(margin - dist, min=0)**2

for epoch in range(1):
    for img1, img2, label in trainloader:
        img1, img2, label = img1.to(device), img2.to(device), label.to(device)
        dist = model(img1, img2)
        loss = contrastive_loss(dist, label).mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch Loss: {loss.item():.4f}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.0 MB/s eta 0:00:00


100%|██████████| 9.91M/9.91M [00:00<00:00, 60.3MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.72MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 13.2MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.39MB/s]


Epoch Loss: 0.2562
